In [43]:
import numpy as np
from dmrghandler.src.dmrghandler.qchem_dmrg_calc import single_qchem_dmrg_calc
from dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
import dmrghandler.src.dmrghandler.dmrg_calc_prepare as dmrg_calc_prepare
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from block2 import SZ as block2_SZ
from get_dmrg_config import get_dmrg_process_param, get_dmrg_config
from pathlib import Path
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from pyblock2._pyscf.ao2mo import integrals as itg
import pyscf
import scipy
from basic_definitions import chemist_to_physicist, get_ferm_op, physicist_to_chemist
from Hamiltonians.molecular_hamiltonians import get_n2, get_h2o, get_hf, get_he, get_co, get_h2
import openfermion as of
from Hamiltonians.dmrg_hamiltonians import get_dmrg_hamiltonian, dmrg_non_exact

In [44]:
from pyscf import gto, scf, cc

mol = gto.M(atom="N 0 0 0; N 0 0 1.4", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
mycc = mf.CISD().run()
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
                                                                     ncore=0,
                                                                     g2e_symm=1)

In [45]:
mf = scf.HF(mol).run()
# Note that the line following these comments could be replaced by
# mycc = cc.CCSD(mf)
# mycc.kernel()
mycc = cc.CCSD(mf).run()
et = mycc.ccsd_t()
print('CCSD(T) total energy', mycc.e_tot + et)

CCSD(T) total energy -107.61443502118154


In [46]:
mycc.kernel()[2].shape

(7, 7, 3, 3)

In [47]:
mycc.t1

array([[ 0.00000000e+00,  0.00000000e+00, -2.61783400e-05],
       [ 0.00000000e+00,  0.00000000e+00,  5.21831251e-18],
       [ 0.00000000e+00,  0.00000000e+00,  6.89781774e-17],
       [ 0.00000000e+00,  0.00000000e+00,  1.95234847e-02],
       [ 0.00000000e+00,  0.00000000e+00, -5.39881196e-15],
       [ 0.00000000e+00,  3.17074522e-15,  0.00000000e+00],
       [ 1.88044210e-15,  0.00000000e+00,  0.00000000e+00]])

# Construct $e^{-T}$

In [48]:
base_t1 = np.zeros((ncas, ncas))
for i in range(len(mycc.t1)):
    for j in range(len(mycc.t1[i])):
        base_t1[j, i] = mycc.t1[i, j]

base_t2 = np.zeros((ncas, ncas, ncas, ncas))

for i in range(len(mycc.t2)):
    for j in range(len(mycc.t2[i])):
        for k in range(len(mycc.t2[i][j])):
            for l in range(len(mycc.t2[i][j][k])):
                base_t2[k, l, i, j] = mycc.t2[i, j, k, l]
    
print(base_t1.shape)
print(base_t2.shape)

(10, 10)
(10, 10, 10, 10)


In [49]:
exp_t = scipy.linalg.expm(base_t1 + base_t2)
exp_tn = scipy.linalg.expm(- base_t2 - base_t1)

# Construct $\tilde{H} = e^{-T}\hat{H}e^{T}$

In [50]:
### Given h1e, g2e, the following returns the physicist 
absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e.copy()), np.array(g2e.copy()), norb=ncas, nelec=n_elec, fac=0.5)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), ncas).astype(np.array(h1e.copy()).dtype, copy=False)

In [51]:
H_tilde = exp_tn @ h_in_tbt @ exp_t

In [52]:
print(H_tilde)

[[[[-9.41567340e-01  1.34285530e-16 -1.51732077e-17 ...  3.66610237e-32
    -3.05141095e-50 -4.02106361e-02]
   [ 2.88977214e-06 -9.40992986e-01 -7.03213992e-02 ...  7.85742549e-41
     5.54566500e-32 -5.29403887e-08]
   [ 1.55136533e-03 -7.24906648e-02 -1.01120617e+00 ...  3.68796266e-37
     1.70458054e-35 -1.73891914e-04]
   ...
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -9.77029707e-01
     0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    -9.77029707e-01  0.00000000e+00]
   [-4.02042327e-02 -5.20501741e-17  3.12670924e-17 ...  0.00000000e+00
     0.00000000e+00 -9.40622085e-01]]

  [[ 2.47034040e-06  9.38949347e-01 -9.43658156e-02 ... -2.90596729e-16
    -1.73191142e-17  1.49010281e-17]
   [ 9.38949347e-01 -1.27177405e-16  1.12886634e-17 ... -2.27983888e-17
    -4.89995511e-16 -5.64251231e-02]
   [-9.43657539e-02  1.96898162e-03 -2.35686157e-03 ... -3.80015154e-21
    -1.51072595e-22  9.18900438e-02]
   ...
   

In [54]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)

one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, h_in_tbt)

In [66]:
import shutil
import os

# Define the directory path
dir_path = "./mps_directory"
dir_path2 = "./mps_directory2"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")
    
if os.path.exists(dir_path2):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path2)
    print(f"Deleted '{dir_path2}' and all its contents.")
else:
    print(f"Directory '{dir_path2}' does not exist.")

Directory './mps_directory' does not exist.
Deleted './mps_directory2' and all its contents.


In [55]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       6038

Build MPO | Nsites =    10 | Nterms =       6038 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.001 T = 0.008
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      279 SPT = 0.8591 Tmvc = 0.001 T = 0.002
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      483 SPT = 0.9335 Tmvc = 0.000 T = 0.002
 Site =     3 /    10 .. Mmpo =   146 DW = 0.00e+00 NNZ =     2591 SPT = 0.8387 Tmvc = 0.000 T = 0.003
 Site =     4 /    10 .. Mmpo =   118 DW = 0.00e+00 NNZ =     1035 SPT = 0.9399 Tmvc = 0.000 T = 0.002
 Site =     5 /    10 .. Mmpo =   114 DW = 0.00e+00 NNZ =      407 SPT = 0.9697 Tmvc = 0.000 T = 0.001
 Site =     6 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =      411 SPT = 0.9714 Tmvc = 0.000 T = 0.001
 Site =     7 /    10 .. Mmpo =    74 DW = 0.00e+00 NNZ =      279 SPT = 0.9701 Tmvc = 0.000 T = 0.001
 Site =     8 /    10 .. Mmpo =

In [67]:
one_body_dummy = np.zeros((ncas, ncas))
obt2, tbt2 = chemist_to_physicist(one_body_dummy, H_tilde)

In [68]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)


In [69]:
loop_results2 = dmrg_central_loop(obt2, tbt2, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory2", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       9982

Build MPO | Nsites =    10 | Nterms =       9982 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /    10 .. Mmpo =    82 DW = 0.00e+00 NNZ =      399 SPT = 0.8378 Tmvc = 0.001 T = 0.003
 Site =     2 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =      838 SPT = 0.9189 Tmvc = 0.001 T = 0.003
 Site =     3 /    10 .. Mmpo =   178 DW = 0.00e+00 NNZ =     1715 SPT = 0.9235 Tmvc = 0.001 T = 0.004
 Site =     4 /    10 .. Mmpo =   200 DW = 0.00e+00 NNZ =     3913 SPT = 0.8901 Tmvc = 0.001 T = 0.005
 Site =     5 /    10 .. Mmpo =   160 DW = 0.00e+00 NNZ =      862 SPT = 0.9731 Tmvc = 0.000 T = 0.002
 Site =     6 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =      891 SPT = 0.9558 Tmvc = 0.000 T = 0.003
 Site =     7 /    10 .. Mmpo =    76 DW = 0.00e+00 NNZ =      353 SPT = 0.9631 Tmvc = 0.000 T = 0.001
 Site =     8 /    10 .. Mmpo =

KeyboardInterrupt: 